In [1]:
#pip install tensorflow==2.2.0

In [2]:
from loaders.Loaders import CSVLoader
from compoundFeaturization.rdkitFingerprints import MorganFingerprint
from splitters.splitters import SingletaskStratifiedSplitter

from sklearn.ensemble import RandomForestRegressor
from models.sklearnModels import SklearnModel
from metrics.Metrics import Metric
from metrics.metricsFunctions import r2_score, mean_absolute_error, mean_squared_error, median_absolute_error

import tensorflow as tf
print(tf.version.VERSION)

from parameterOptimization.HyperparameterOpt import HyperparamOpt_Valid, HyperparamOpt_CV


2.2.0


In [3]:
#Load Dataset
dataset = CSVLoader(dataset_path='data/PC-3.csv', 
                    mols_field='smiles', 
                    labels_fields='pIC50')
dataset = dataset.create_dataset()
dataset.get_shape()

Mols_shape:  4294
Features_shape:  X not defined!
Labels_shape:  (4294,)


In [4]:
#Featurization
dataset = MorganFingerprint().featurize(dataset)
dataset.get_shape()

Featurizing datapoint 0
Featurizing datapoint 1000
Featurizing datapoint 2000
Featurizing datapoint 3000
Featurizing datapoint 4000
Mols_shape:  4294
Features_shape:  (4294, 1024)
Labels_shape:  (4294,)


In [5]:
#Data Split
splitter = SingletaskStratifiedSplitter()
train_dataset, valid_dataset, test_dataset = splitter.train_valid_test_split(dataset=dataset, frac_train=0.6, 
                                                                             frac_valid=0.2, frac_test=0.2)

train_dataset.get_shape()
valid_dataset.get_shape()
test_dataset.get_shape()

Mols_shape:  2574
Features_shape:  (2574, 1024)
Labels_shape:  (2574,)
Mols_shape:  858
Features_shape:  (858, 1024)
Labels_shape:  (858,)
Mols_shape:  858
Features_shape:  (858, 1024)
Labels_shape:  (858,)


In [6]:
#Scikit-Learn Random Forest
#rf = RandomForestRegressor()
#model = SklearnModel(model=rf)

In [7]:
from models.kerasModels import KerasModel
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


input_dim = train_dataset.X.shape[1]
print(input_dim)

def build_model():
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=[1024]))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1))
    
    optimizer = tf.keras.optimizers.RMSprop(0.001)

    model.compile(loss='mse', optimizer=optimizer, metrics=['mae', 'mse'])
    return model

model = KerasModel(build_model, mode='regression', epochs = 5, verbose=1)

1024


In [8]:
#cross validation
model.cross_validate(dataset, Metric(r2_score), folds=3)

Computing K-fold split
Epoch 1/5
287/287 [==============================] - 0s 1ms/step - loss: 1.5568 - mae: 0.8537 - mse: 1.5568
Epoch 2/5
287/287 [==============================] - 0s 1ms/step - loss: 0.5002 - mae: 0.5350 - mse: 0.5002
Epoch 3/5
287/287 [==============================] - 0s 1ms/step - loss: 0.3706 - mae: 0.4556 - mse: 0.3706
Epoch 4/5
287/287 [==============================] - 0s 1ms/step - loss: 0.2912 - mae: 0.4022 - mse: 0.2912
Epoch 5/5
287/287 [==============================] - 0s 1ms/step - loss: 0.2527 - mae: 0.3729 - mse: 0.2527
Train Score: 
287/287 [==============================] - 0s 686us/step
r2_score: 
 0.8016884665640229
Test Score: 
144/144 [==============================] - 0s 710us/step
r2_score: 
 -0.10019472535757368
Epoch 1/5
287/287 [==============================] - 0s 2ms/step - loss: 1.5148 - mae: 0.8609 - mse: 1.5148
Epoch 2/5
287/287 [==============================] - 0s 2ms/step - loss: 0.5372 - mae: 0.5548 - mse: 0.5372
Epoch 3/5
287/28

(None,
 0,
 0,
 [0.8016884665640229, 0.8541621609522112, 0.7713730553153497],
 [-0.10019472535757368, -0.19253639315276816, -0.527210656178245],
 0.8090745609438613,
 -0.2733139248961956)

In [9]:
# model training
model.fit(train_dataset)

kerasModels fit!
Epoch 1/5
258/258 [==============================] - 0s 1ms/step - loss: 1.8194 - mae: 0.9414 - mse: 1.8194
Epoch 2/5
258/258 [==============================] - 0s 2ms/step - loss: 0.5914 - mae: 0.5827 - mse: 0.5914
Epoch 3/5
258/258 [==============================] - 0s 2ms/step - loss: 0.4357 - mae: 0.5059 - mse: 0.4357
Epoch 4/5
258/258 [==============================] - 0s 2ms/step - loss: 0.3464 - mae: 0.4432 - mse: 0.3464
Epoch 5/5
258/258 [==============================] - 0s 1ms/step - loss: 0.2856 - mae: 0.4016 - mse: 0.2856


In [10]:
metrics = [Metric(mean_absolute_error), Metric(mean_squared_error), Metric(median_absolute_error), Metric(r2_score)]
print("#############################")
# evaluate the model
print('Training Dataset: ')
train_score = model.evaluate(train_dataset, metrics)
print("#############################")
print('Validation Dataset: ')
valid_score = model.evaluate(valid_dataset, metrics)
print("#############################")
print('Test Dataset: ')
test_score = model.evaluate(test_dataset, metrics)


#############################
Training Dataset: 
<class 'tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor'>
258/258 [==============================] - 0s 713us/step
mean_absolute_error: 
 0.3638539517295209
mean_squared_error: 
 0.22843930288214348
median_absolute_error: 
 0.29652029991149886
r2_score: 
 0.7844462356460913
#############################
Validation Dataset: 
<class 'tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor'>
86/86 [==============================] - 0s 700us/step
mean_absolute_error: 
 0.5703043667593982
mean_squared_error: 
 0.5865894145582745
median_absolute_error: 
 0.4531481742858885
r2_score: 
 0.44466672789142125
#############################
Test Dataset: 
<class 'tensorflow.python.keras.wrappers.scikit_learn.KerasRegressor'>
86/86 [==============================] - 0s 672us/step
mean_absolute_error: 
 0.573556881925354
mean_squared_error: 
 0.6206730046075947
median_absolute_error: 
 0.4418857192993162
r2_score: 
 0.4136421455414960

In [15]:
#Build a model function for hyperparameter optimization
def rf_model_builder(n_estimators=10, max_features='auto', criterion='mse'):
    rf_model = RandomForestRegressor(n_estimators=n_estimators, max_features=max_features,
                                     criterion=criterion)
    return rf_model

params_dict_rf = {"n_estimators": [10, 100],
                  "max_features": ["auto", "sqrt", "log2", None],
                  "criterion": ["mse", "mae"]
                  }
    
model = SklearnModel(rf_model_builder, 'regression')

In [12]:
#Hyperparameter Optimization
#optimizer = HyperparamOpt_Valid(rf_model_builder)

#best_rf, best_hyperparams, all_results = optimizer.hyperparam_search(params_dict_rf, 
#                                                                     train_dataset, 
#                                                                     valid_dataset, 
#                                                                     Metric(r2_score),
#                                                                     n_iter_search=15)

#print('#################')
#print(best_hyperparams)
#print(best_rf)

In [13]:
#Evaluate model
#best_rf.evaluate(test_dataset, metrics)

In [18]:
#Hyperparameter Optimization with CV
optimizer = HyperparamOpt_CV(rf_model_builder)

best_rf, best_hyperparams, all_results = optimizer.hyperparam_search('sklearn',
                                                                     params_dict_rf, 
                                                                     train_dataset,  
                                                                     'r2', 
                                                                     cv=3,
                                                                     n_iter_search=10,
                                                                    n_jobs=8)

print('#################')
print(best_hyperparams)
print(best_rf)

#Evaluate model
best_rf.evaluate(test_dataset, metrics)

MODEL TYPE:  sklearn
asdasdas
Fitting 10 random models from a space of 16 possible models.

 
 Best r2: -10.981018 using {'n_estimators': 100, 'max_features': 'sqrt', 'criterion': 'mse'}

 r2: -12.113956 (9.600818) with: {'n_estimators': 10, 'max_features': 'log2', 'criterion': 'mae'} 


 r2: -10.989558 (9.215672) with: {'n_estimators': 100, 'max_features': 'log2', 'criterion': 'mse'} 


 r2: -11.252557 (9.494626) with: {'n_estimators': 100, 'max_features': 'log2', 'criterion': 'mae'} 


 r2: -10.981018 (8.888105) with: {'n_estimators': 100, 'max_features': 'sqrt', 'criterion': 'mse'} 


 r2: -11.425314 (9.029259) with: {'n_estimators': 100, 'max_features': None, 'criterion': 'mse'} 


 r2: -12.530082 (9.692693) with: {'n_estimators': 10, 'max_features': 'sqrt', 'criterion': 'mae'} 


 r2: -12.069208 (9.710852) with: {'n_estimators': 100, 'max_features': None, 'criterion': 'mae'} 


 r2: -11.196103 (8.330170) with: {'n_estimators': 10, 'max_features': 'log2', 'criterion': 'mse'} 


 r2

{'mean_absolute_error': 0.45967412684937947,
 'mean_squared_error': 0.41406634541935244,
 'median_absolute_error': 0.3373499999999976,
 'r2_score': 0.6088261417828797}